In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook

def load_stock_data(file_path):
    wb = load_workbook(filename=file_path, read_only=True)
    all_data = {}
    
    for sheet_name in wb.sheetnames:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
        all_data[sheet_name] = df
        
    return all_data

def calculate_signals(df):
    df['55d_high'] = df['High'].rolling(window=18).max()
    df['20d_low'] = df['Low'].rolling(window=11).min()
    
    # Entry signal (crosses above 18-day high)
    df['entry_signal'] = (df['Close'] > df['55d_high'].shift(1)).astype(int)
    
    # Exit signal (crosses below 11-day low)
    df['exit_signal'] = (df['Close'] < df['20d_low'].shift(1)).astype(int)
    
    return df

def backtest_strategy(stock_data, initial_capital=100000):
    results = {}
    
    for stock_name, df in stock_data.items():
        df = calculate_signals(df)
        
        position = 0
        capital_remaining = initial_capital
        entry_prices = []
        daily_pnl = []
        
        for i in range(1, len(df)):
            close_price = df['Close'].iloc[i]
            
            # Check exit signal first
            if df['exit_signal'].iloc[i] == 1 and position > 0:
                # Exit all positions
                pnl = position * (close_price - np.mean(entry_prices))
                capital_remaining += position * close_price
                position = 0
                entry_prices = []
            
            # Check entry signal
            elif df['entry_signal'].iloc[i] == 1 and capital_remaining >= initial_capital/5:
                # Enter new position with 1/5 of initial capital
                new_position = (initial_capital/5) // close_price
                if new_position > 0:
                    position += new_position
                    capital_remaining -= new_position * close_price
                    entry_prices.extend([close_price] * int(new_position))
            
            # Calculate daily PNL
            if position > 0:
                daily_pnl.append(position * (close_price - np.mean(entry_prices)))
            else:
                daily_pnl.append(0)
        
        df['Daily_PNL'] = [0] + daily_pnl
        df['Cumulative_PNL'] = df['Daily_PNL'].cumsum()
        
        results[stock_name] = df
        
    return results

def calculate_portfolio_metrics(results):
    # Combine all PNLs
    combined_pnl = pd.DataFrame()
    
    for stock_name, df in results.items():
        if combined_pnl.empty:
            combined_pnl['Date'] = df['Date']
            combined_pnl['Total_PNL'] = df['Cumulative_PNL']
        else:
            combined_pnl['Total_PNL'] += df['Cumulative_PNL']
    
    # Calculate drawdown
    combined_pnl['Peak'] = combined_pnl['Total_PNL'].expanding().max()
    combined_pnl['Drawdown'] = (combined_pnl['Total_PNL'] - combined_pnl['Peak']) / combined_pnl['Peak'] * 100
    
    return combined_pnl

def plot_results(combined_pnl):
    plt.figure(figsize=(15, 10))
    
    # Plot PNL
    plt.subplot(2, 1, 1)
    plt.plot(combined_pnl['Date'], combined_pnl['Total_PNL'])
    plt.title('Combined PNL Over Time')
    plt.xlabel('Date')
    plt.ylabel('PNL (Rs.)')
    plt.grid(True)
    
    # Plot Drawdown
    plt.subplot(2, 1, 2)
    plt.fill_between(combined_pnl['Date'], combined_pnl['Drawdown'], 0, color='red', alpha=0.3)
    plt.plot(combined_pnl['Date'], combined_pnl['Drawdown'], color='red')
    plt.title('Portfolio Drawdown')
    plt.xlabel('Date')
    plt.ylabel('Drawdown (%)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

def main():
    # Load data
    file_path = 'RIL.xlsx'
    stock_data = load_stock_data(file_path)
    
    # Run backtest
    results = backtest_strategy(stock_data)
    
    # Calculate portfolio metrics
    combined_metrics = calculate_portfolio_metrics(results)
    
    # Plot results
    plot_results(combined_metrics)
    
    # Print summary statistics
    final_pnl = combined_metrics['Total_PNL'].iloc[-1]
    max_drawdown = combined_metrics['Drawdown'].min()
    print(f"Final Portfolio PNL: Rs. {final_pnl:,.2f}")
    print(f"Maximum Drawdown: {max_drawdown:.2f}%")
    print(f"Total Return: {final_pnl / 100000 * 100:.2f}%")

if __name__ == "__main__":
    main()